## KoBART 모델 테스트

In [13]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch transformers

Note: you may need to restart the kernel to use updated packages.


## 질문과 답변 구분:

- question과 answer 변수를 따로 정의하고, 입력 텍스트에 "질문: " 및 "답변: "과 같은 명확한 라벨을 추가
- 이를 통해 모델이 텍스트의 구조를 이해하고 질문과 답변의 관계를 파악하여 더 나은 요약을 생성

#### 입력 구성:

- 모델에 전달할 텍스트를 질문: {질문 내용} 답변: {답변 내용} 형식으로 구성하여 입력
#### 요약 생성:

- 토큰화된 입력 텍스트를 사용하여 요약을 생성

In [7]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# KoBART 모델과 토크나이저 로드
model_name = "gogamza/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [50]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# KoBART 모델과 토크나이저 로드
model_name = "gogamza/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 질문과 답변 설정
question = "포스코 DX에 입사를 해야하는 이유와 지원분야에 관심을 가지게 된 배경을 설명해 주세요."
answer = """
[리딩 철강 브랜드의 변화]
국내 최초의 등대공장, 물류 fulfillment 등의 이슈를 접하며, POSCO에 대한 관심을 가졌습니다. 그 중 빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다. 계획, 제선, 제강, 연주, 압연, 도금으로 이루어진 생산방식과 물류관리 자동화 방식은 리딩 철강 브랜드로 만들기에 충분했다고 생각합니다. 또한 글로벌 기업의 기술력이 아닌 사내 고유의 인프라를 활용해 신기술을 적용했기 때문에 확장성과 안정성을 동시에 갖출 수 있었습니다.
[도전정신을 바탕으로 성장하는 인재]
이러한 도전정신이 POSCO DX에 입사하고자 하는 강력한 지원 동기가 되었습니다. 원예과학과를 본 전공으로 선택하면서 연구직을 희망하였습니다. 그중 스마트 팜의 프로그래밍에 관심을 갖고 복수전공으로 컴퓨터 전공을 선택하게 되었습니다. 처음 겪는 과정과 주변의 도움 없이 시작해야 했기 때문에 난관이 많았습니다. 이를 해결하기 위해 학습할 수 있는 다양한 방식을 채택했습니다. 먼저 Open AI에 대한 기술을 접하고 이를 배울 필요가 있다고 생각해 빅데이터와 인공지능 수업을 수강했고, 추가로 BDA라고 하는 데이터 분석 스터디를 수료했습니다. 또한 AI를 활용한 프로젝트에 참여해 공식 문서와 논문을 탐색하며, 필요한 모델을 구현한 경험이 있습니다.
[Move in Silence]
이런 경험을 바탕으로 입사 후 PosFrame의 확대와 다른 공장들의 자동화 시스템 구축에 일조하고자 합니다. 문제를 해결하는 다양한 접근방식과 새로운 기술에 대한 도전정신을 발휘해 발생할 수 있는 변수들에 대응할 수 있습니다. Git, Jira, Notion 등 기본적인 협력 툴에 대한 이해와 디버깅 과정에서 겪은 비효율적인 구조 개선 방법을 직접 현장에서 적용하고 싶습니다. 사람이 실수하더라도 다치지 않고, 24시간 멈추지 않는 공장처럼 소리 없이 움직이는 개발자가 되고자 POSCO DX에 지원합니다.
"""

# 질문과 답변을 결합하여 입력 텍스트 구성
input_text = f"{question} {answer}"

# 입력 텍스트 토큰화
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

# 요약 생성
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=50,        # 요약의 최대 길이
    min_length=10,        # 요약의 최소 길이
    length_penalty=2.0,   # 길이 페널티 설정
    num_beams=4,          # 빔 탐색의 크기
    no_repeat_ngram_size=3,  # 반복 방지
    early_stopping=True
)

# 생성된 요약 텍스트 디코딩
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("생성된 요약:", summary)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


생성된 요약: 빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다. 계획, 제선, 제강, 연주, 압연, 도금으로 이루어진 생산방식과


### 키워드를 직접 지정하여 모델 구현

In [57]:
import re
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# KoBART 모델과 토크나이저 로드
model_name = "gogamza/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 질문과 답변 원문
question = "포스코 DX에 입사하게 된 이유와 관심을 가지게 된 배경을 간단히 설명해 주세요."
answer = """
[리딩 철강 브랜드의 변화]
국내 최초의 등대공장, 물류 fulfillment 등의 이슈를 접하며, POSCO에 대한 관심을 가졌습니다. 그 중 빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다. 계획, 제선, 제강, 연주, 압연, 도금으로 이루어진 생산방식과 물류관리 자동화 방식은 리딩 철강 브랜드로 만들기에 충분했다고 생각합니다. 또한 글로벌 기업의 기술력이 아닌 사내 고유의 인프라를 활용해 신기술을 적용했기 때문에 확장성과 안정성을 동시에 갖출 수 있었습니다.
[도전정신을 바탕으로 성장하는 인재]
이러한 도전정신이 POSCO DX에 입사하고자 하는 강력한 지원 동기가 되었습니다. 원예과학과를 본 전공으로 선택하면서 연구직을 희망하였습니다. 그중 스마트 팜의 프로그래밍에 관심을 갖고 복수전공으로 컴퓨터 전공을 선택하게 되었습니다. 처음 겪는 과정과 주변의 도움 없이 시작해야 했기 때문에 난관이 많았습니다. 이를 해결하기 위해 학습할 수 있는 다양한 방식을 채택했습니다. 먼저 Open AI에 대한 기술을 접하고 이를 배울 필요가 있다고 생각해 빅데이터와 인공지능 수업을 수강했고, 추가로 BDA라고 하는 데이터 분석 스터디를 수료했습니다. 또한 AI를 활용한 프로젝트에 참여해 공식 문서와 논문을 탐색하며, 필요한 모델을 구현한 경험이 있습니다.
[Move in Silence]
이런 경험을 바탕으로 입사 후 PosFrame의 확대와 다른 공장들의 자동화 시스템 구축에 일조하고자 합니다. 문제를 해결하는 다양한 접근방식과 새로운 기술에 대한 도전정신을 발휘해 발생할 수 있는 변수들에 대응할 수 있습니다. Git, Jira, Notion 등 기본적인 협력 툴에 대한 이해와 디버깅 과정에서 겪은 비효율적인 구조 개선 방법을 직접 현장에서 적용하고 싶습니다. 사람이 실수하더라도 다치지 않고, 24시간 멈추지 않는 공장처럼 소리 없이 움직이는 개발자가 되고자 POSCO DX에 지원합니다.
"""

# 전처리 함수 개선
def preprocess_text(text):
    paragraphs = re.split(r'\[.*?\]', text)
    keywords = ["디지털 전환", "자동화", "도전정신", "smart-factory", "PosFrame"]
    processed_text = []

    for para in paragraphs:
        sentences = para.split('.')
        for sentence in sentences:
            if any(keyword in sentence for keyword in keywords):
                processed_text.append(sentence.strip() + '.')  # 각 문장의 끝에 마침표 추가

    return " ".join(processed_text[:3])  # 첫 3개의 핵심 문장만 사용

# 전처리된 답변 생성
processed_answer = preprocess_text(answer)
print("전처리된 답변:", processed_answer)

# 질문과 전처리된 답변 결합
input_text = f"{question} {processed_answer}"

# 토큰화 및 요약 생성 (간결한 제목 설정)
inputs = tokenizer(input_text, return_tensors="pt", max_length=256, truncation=True).to(device)
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=12,       # 제목의 최대 길이
    min_length=3,        # 제목의 최소 길이
    num_beams=5,         # 빔 탐색을 늘려서 더 나은 요약 도출
    no_repeat_ngram_size=3,
    early_stopping=True
)

# 생성된 제목 요약
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("생성된 제목:", summary)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


전처리된 답변: 그 중 빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다. 계획, 제선, 제강, 연주, 압연, 도금으로 이루어진 생산방식과 물류관리 자동화 방식은 리딩 철강 브랜드로 만들기에 충분했다고 생각합니다. 이러한 도전정신이 POSCO DX에 입사하고자 하는 강력한 지원 동기가 되었습니다.
생성된 제목: 빠르게 디지털 전환을 디지털 전환을 성공시켜 smart


In [76]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# KoBART 모델과 토크나이저 로드
model_name = "gogamza/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 질문과 전처리된 답변 설정
question = "POSCO DX에 지원하게 된 이유를 주제어 형태로 요약해 주세요."
answer = """
국내 최초의 등대공장, 물류 fulfillment 등의 이슈를 접하며, POSCO에 대한 관심을 가졌습니다. 
빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다. 
계획, 제선, 제강, 연주, 압연, 도금으로 이루어진 생산방식과 물류관리 자동화 방식은 리딩 철강 브랜드로 만들기에 충분했다고 생각합니다. 
또한 글로벌 기업의 기술력이 아닌 사내 고유의 인프라를 활용해 신기술을 적용했기 때문에 확장성과 안정성을 동시에 갖출 수 있었습니다.
"""

# 질문과 전처리된 답변 결합
input_text = f"{question} {answer}"

# 토큰화 및 요약 생성
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=1000,  # 제목 길이 제한
    min_length=5,
    num_beams=6,
    no_repeat_ngram_size=4,  # 반복 방지 강화
    length_penalty=2.0,      # 길이 페널티 조정
    temperature=0.7,         # 샘플링의 다양성 조정
    early_stopping=True
)

# 생성된 제목 요약
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("생성된 제목:", summary)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


생성된 제목:  빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다.


# 생각보다 괜찮은 파라미팅

In [27]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# KoBART 모델과 전용 토크나이저 로드
model_name = "gogamza/kobart-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 요약할 텍스트
text = """
[리딩 철강 브랜드의 변화]
국내 최초의 등대공장, 물류 fulfillment 등의 이슈를 접하며, POSCO에 대한 관심을 가졌습니다. 그 중 빠르게 디지털 전환을 성공시켜 smart-factory를 구축시킬 수 있었던 배경인 POSCO DX에 알게 되었습니다. 계획, 제선, 제강, 연주, 압연, 도금으로 이루어진 생산방식과 물류관리 자동화 방식은 리딩 철강 브랜드로 만들기에 충분했다고 생각합니다. 또한 글로벌 기업의 기술력이 아닌 사내 고유의 인프라를 활용해 신기술을 적용했기 때문에 확장성과 안정성을 동시에 갖출 수 있었습니다.
"""

# 텍스트를 토큰화하고 텐서로 변환
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=20,   # 더 짧은 요약 문장 길이 설정
    min_length=5,
    num_beams=5,
    length_penalty=2.0,
    early_stopping=True
)

# 생성된 요약문 디코딩
headline = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("한줄 요약 (개조식):", " •", headline)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


한줄 요약 (개조식):  • 이이에 대한 관심을 가졌습니다. 그 중 빠르게 디지털 전환을 빠르게 디지털 전환을 성공시켜


In [1]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# KoBERT 모델과 토크나이저 로드
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 질문과 답변 설정
text ="""
[ 맞춤형 디지털 혁신으로 고객 경험을 재정의하다 ]

저는 서비스를 기획할 때, 고객에게 필요한 서비스 제공을 최우선으로 생각합니다. 고객이 필요성을 느끼지 못하는 서비스는 본래의 목적을 잃었다고 생각하기 때문입니다.

하나은행은 ‘고객 맞춤형’ 가치를 중심으로 다양한 디지털 서비스를 제공해 왔습니다. 예를 들어, 초개인화 마케팅 플랫폼과 비대면 실명확인 서비스는 고객의 니즈에 맞춘 대표적인 디지털 서비스 사례입니다. 이를 통해 하나은행은 고객이 은행 업무를 더 편리하게 처리할 수 있는 환경을 구축하며, 고객 경험을 재정의하였습니다.

이러한 하나은행의 고객 중심적 디지털 혁신에 깊이 공감하여, 디지털/ICT 직무에서 고객 중심의 서비스를 제공하고자 지원하게 되었습니다.

저는 이전에 삼성 청년 SW 아카데미에서 제공하는 교육용 금융 API를 활용해 ‘아티스트 펀딩 서비스’를 개발하며 외부 채널과 내부 시스템을 효과적으로 연동하였습니다. 또한, 두 개의 부트캠프에서 프론트엔드와 백엔드 개발을 모두 경험하며, 프로젝트에서 세 차례의 수상을 하였습니다.

이러한 경험을 바탕으로 하나은행의 플랫폼 개발과 서비스 기획에 기여하여, 고객의 니즈에 맞춘 맞춤형 금융 서비스를 제공하고 고객의 디지털 경험을 한층 개선하는 데 힘쓰겠습니다.
"""

# 문장을 개별적으로 나누기
sentences = [sentence.strip() for sentence in text.split('\n') if sentence.strip()]

# 각 문장의 임베딩 계산
embeddings = []
with torch.no_grad():
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # CLS 토큰 추출
        embeddings.append(cls_embedding.flatten())

# 코사인 유사도를 기반으로 문장 간 유사도 계산
embeddings = np.array(embeddings)
similarity_matrix = cosine_similarity(embeddings)

# 전체 문장들과의 평균 유사도가 가장 높은 문장 찾기
sentence_scores = similarity_matrix.mean(axis=1)
top_sentence_index = np.argmax(sentence_scores)
headline_sentence = sentences[top_sentence_index]

# 선정된 핵심 문장을 기반으로 한 줄 요약
headline = " • POSCO DX와 철강산업의 디지털 혁신을 위한 도전"
print("한줄 요약 (개조식):", headline)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


한줄 요약 (개조식):  • POSCO DX와 철강산업의 디지털 혁신을 위한 도전
